#### This notebook seeks the best combination of Keras model optimizer and loss used in the fit function.
__Accuracy is the goal__

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

datapath = "./train_data_00.npz"

In [2]:
with np.load(datapath) as file:
    labels = file['label']
    spikes = file['spike']

from sklearn.utils import shuffle
labels, spikes = shuffle(labels, spikes)

sample_percentage = 0.4
cut = int(sample_percentage*len(labels))
labels = labels[:cut]
spikes = spikes[:cut]

split = int(len(labels) * 0.8)
train_labels = labels[:split]
train_spikes = spikes[:split]
test_labels = labels[split:]
test_spikes = spikes[split:]

In [3]:
spike_length = np.shape(train_spikes)[1]
hidden_layer_size = spike_length

layers = [
    tf.keras.layers.Dense(spike_length),
    tf.keras.layers.Dense(hidden_layer_size),
    tf.keras.layers.Dense(len(np.unique(labels)))
]

In [4]:
optimizers = np.array([
    "Adadelta",
    "Adagrad",
    "Adam",
    "Adamax",
    "Ftrl",
    "Nadam",
    "RMSprop",
    "SGD"
])

losses = np.array([
    "BinaryCrossentropy",
    "BinaryFocalCrossentropy",
    # "CategoricalCrossentropy",
    "CategoricalHinge",
    # "CosineSimilarity",
    "Hinge",
    "Huber",
    "KLDivergence",
    "LogCosh",
    "MeanAbsoluteError",
    "MeanAbsolutePercentageError",
    "MeanSquaredError",
    "MeanSquaredLogarithmicError",
    "Poisson",
    "SparseCategoricalCrossentropy",
    "SquaredHinge"
])

In [5]:
accuracies = np.ndarray((len(optimizers), len(losses)))
for i, optimizer in tqdm(enumerate(optimizers)):
    for j, loss in enumerate(losses):
        model = tf.keras.Sequential(layers)
        model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=["accuracy"]
        )
        callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=1)
        model.fit(train_spikes, train_labels, epochs=6, callbacks=callback, verbose=0)
        model_loss, model_acc = model.evaluate(test_spikes, test_labels, verbose=0)
        accuracies[i][j] = model_acc

0it [00:00, ?it/s]


ValueError: in user code:

    File "c:\Users\jiyug\Unsynched_Docs\MiV\venv\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\jiyug\Unsynched_Docs\MiV\venv\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\jiyug\Unsynched_Docs\MiV\venv\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\jiyug\Unsynched_Docs\MiV\venv\lib\site-packages\keras\engine\training.py", line 894, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "c:\Users\jiyug\Unsynched_Docs\MiV\venv\lib\site-packages\keras\engine\training.py", line 987, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "c:\Users\jiyug\Unsynched_Docs\MiV\venv\lib\site-packages\

In [ ]:
best_idx = np.argmax(accuracies)
best_opt_idx = int(best_idx / len(losses))
best_loss_idx = best_idx % len(losses)
print("Most accurate combination = ", optimizers[best_opt_idx], "+", losses[best_loss_idx])
print("with accuracy = ", accuracies[best_opt_idx][best_loss_idx])

Most accurate combination =  Adam + Hinge
with accuracy =  0.8013302087783813
